In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

# Definir el modelo
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc = nn.Linear(10, 5)

    def forward(self, x):
        return self.fc(x)

def train(rank, world_size):
    # Inicializar el proceso de comunicación entre los nodos
    dist.init_process_group(backend='nccl', init_method='tcp://127.0.0.1:23456', rank=rank, world_size=world_size)

    # Crear el modelo y moverlo a la GPU asignada a cada proceso
    model = Model().to(rank)
    ddp_model = DDP(model, device_ids=[rank])

    # Definir los datos de entrenamiento
    inputs = torch.randn(100, 10).to(rank)
    targets = torch.randn(100, 5).to(rank)

    # Definir la función de pérdida y el optimizador
    criterion = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.01)

    # Ciclo de entrenamiento
    for epoch in range(5):
        outputs = ddp_model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Calcular la pérdida
        optimizer.zero_grad()
        loss.backward()  # Backward pass
        optimizer.step()  # Actualizar los parámetros

        # Imprimir el progreso de entrenamiento
        if rank == 0:
            print(f'Epoch [{epoch+1}/5], Loss: {loss.item()}')

    # Finalizar el proceso de comunicación
    dist.destroy_process_group()

In [4]:
mp.spawn(train, args=(2,), nprocs=2)  # Entrenamiento en 2 GPUs

Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
  File "/opt/homebrew/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'train' on <module '__main__' (built-in)>
    self = reduction.p

ProcessExitedException: process 0 terminated with exit code 1

En este notebook, se utiliza Distributed Data Parallelism (DDP) para entrenar un modelo en paralelo en dos GPUs. El proceso de entrenamiento se ejecuta en paralelo en diferentes nodos o GPUs, y los gradientes se sincronizan automáticamente entre los nodos.

El código utiliza torch.multiprocessing para crear un proceso por GPU y luego cada proceso se comunica con los demás utilizando dist.init_process_group(). Luego, se define el modelo y se mueve a la GPU asignada a cada proceso. El modelo se envuelve en DDP para habilitar la paralelización de los cálculos. A continuación, se definen los datos de entrenamiento, la función de pérdida y el optimizador.

El bucle de entrenamiento se ejecuta en paralelo en todos los procesos/nodos. Cada proceso realiza el forward pass y el backward pass utilizando su porción de los datos. Los gradientes se sincronizan automáticamente entre los nodos, lo que permite una actualización de los parámetros consistente y paralela. Finalmente, se destruye el grupo de procesos con dist.destroy_process_group().